<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_0/ate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna
import optuna

In [3]:
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [4]:
path="/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/"

In [47]:
test = pd.read_json(path+'ATE_ABSITA_test_set/ate_absita_gold.ndjson',
                    lines=True)

train = pd.read_json(path+'ATE_ABSITA_training_set/ate_absita_training.ndjson',
                     lines=True)

data_sinonimi = pd.read_csv(path+"ATE_ABSITA_test_set/sinonimi.csv")
data_embedding = pd.read_csv(path+"ATE_ABSITA_test_set/embedding.csv")

In [ ]:
print(test.columns)
print(train.columns)
print(data_sinonimi.columns)
print(data_embedding.columns)

In [48]:
train.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
data_sinonimi.drop(columns=['polarities','aspects_position','aspects'], inplace=True)
data_embedding.drop(columns=['polarities','aspects_position','aspects'], inplace=True)
print(f'Contains {len(train)} sentences')
print(f'Contains {len(test)} sentences')
print(f'Contains {len(data_sinonimi)} sentences')
print(f'Contains {len(data_embedding)} sentences')

Contains 3054 sentences
Contains 1200 sentences
Contains 1200 sentences
Contains 1200 sentences


Creazione colonna Positivi/Negativi

In [ ]:
train["review_type"] = train["score"].apply(lambda x: "neg" if x < 5 else "pos")
test["review_type"] = test["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_sinonimi["review_type"] = data_sinonimi["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_embedding["review_type"] = data_embedding["score"].apply(lambda x: "neg" if x < 5 else "pos")

print(f'TRAIN::\n{train.review_type.value_counts()}')
print(f'TEST::\n{test.review_type.value_counts()}')
print(f'SINONIMI::\n{data_sinonimi.review_type.value_counts()}')
print(f'EMBEDDING::\n{data_embedding.review_type.value_counts()}')

Rimozione Colonna Score in quanto non piu significativa per la Sentiment Analysis

In [ ]:
train.drop(columns=['score'], inplace=True)
test.drop(columns=['score'], inplace=True)
data_sinonimi.drop(columns=['score'], inplace=True)
data_embedding.drop(columns=['score'], inplace=True)

In [7]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Trasformazione input da frasi a vettori di parole

In [8]:
def my_text_to_word_sequence(sentence):
  return keras.preprocessing.text.text_to_word_sequence(sentence,
                                                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`\'{|}~\t\n',
                                                        lower=True)

TRAIN, Encoding e Dataset per modello

In [9]:
one_hot_train = OneHotEncoder(sparse=False).fit_transform(
        train.review_type.to_numpy().reshape(-1, 1)
        )

In [16]:
sentences = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

TEST, Encoding e Dataset per modello

In [18]:
one_hot_test = OneHotEncoder(sparse=False).fit_transform(
        test.review_type.to_numpy().reshape(-1, 1)
        )

In [19]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI, Encoding e Dataset per modello

In [50]:
one_hot_sin = OneHotEncoder(sparse=False).fit_transform(
        data_sinonimi.review_type.to_numpy().reshape(-1, 1)
        )

In [51]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING, Encoding e Dataset per modello

In [12]:
one_hot_emb = OneHotEncoder(sparse=False).fit_transform(
        data_embedding.review_type.to_numpy().reshape(-1, 1)
        )

In [13]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [20]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)


print(f'Il massimo è {max}')

Il massimo è 85


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [ ]:
embedded_trainset = np.zeros(shape=(len(sentences), max, 300))
for i, sentence in enumerate(sentences):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [ ]:
embedded_test = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [52]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [44]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

#Model

In [38]:
best_params = optuna.load_study(study_name="ATE", storage="sqlite:///"+path+"optuna_ATE_studio_0.db").best_params

In [39]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params["units"],
                                                             recurrent_dropout=best_params["dropout"],
                                                             activation='tanh')))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adam(0.001),
            metrics=['accuracy'])

In [ ]:
#train model
result = model.fit(embedded_trainset,
                 one_hot_train,
                 epochs=100,
                 batch_size=best_params["batch_size"],
                 callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

In [ ]:
#save the model
model.save_weights(path+'ATE_w_studio_0.h5')

In [41]:
model.load_weights(path+'ATE_w_studio_0.h5')

#EVALUATION

In [ ]:
result_base=model.evaluate(embedded_trainset,one_hot_train,batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

56/56 [==============================] - 6s 46ms/step - loss: 0.0164 - accuracy: 0.9969
DATASET ORIGINARIO[0.01643429324030876, 0.9967256188392639]


In [ ]:
result_base=model.evaluate(embedded_test,one_hot_test,batch_size=best_params['batch_size'],)
print(f'DATASET DEV{result_base}')

22/22 [==============================] - 1s 47ms/step - loss: 0.6068 - accuracy: 0.7142
DATASET DEV[0.6067888140678406, 0.7141666412353516]


In [53]:
result_base=model.evaluate(embedded_sin,one_hot_sin,batch_size=best_params['batch_size'])
print(f'DATASET SINONIMI{result_base}')

22/22 [==============================] - 1s 44ms/step - loss: 1.7832 - accuracy: 0.6433
DATASET SINONIMI[1.7832045555114746, 0.6433333158493042]


In [46]:
result_base=model.evaluate(embedded_emb,one_hot_emb,batch_size=best_params['batch_size'])
print(f'DATASET EMBEDDING{result_base}')

22/22 [==============================] - 1s 40ms/step - loss: 1.6284 - accuracy: 0.6650
DATASET EMBEDDING[1.6283600330352783, 0.6650000214576721]
